# Arma Experiment example

In this example, we will use the ARMA dataset provided by this package, get some attributions,
and evaluate them.

### Imports

In [1]:
import torch as th

from captum.attr import IntegratedGradients
from pytorch_lightning import seed_everything

from tint.datasets import Arma
from tint.metrics.white_box import aur

/Users/joseph.enguehard/opt/miniconda3/envs/tint/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Make reproducible experiment 

For this example, we will make everything reproducible. With this aim, we use the 
tool from Pytorch-Lightning: seed_everything.

In [2]:
seed = 42
seed_everything(seed=seed, workers=True)

Global seed set to 42


42

### Data loading

We load the Arma dataset, and eventually download it (since arma is a synthetic dataset, 
download actually generates the data).

In [3]:
arma = Arma(seed=seed)

In [4]:
arma.download()

For this example, we only use the first 10 processes. We load the input data as well as the true 
attributions. This synthetic example indeed provides this information, allowing us to evaluate 
each attribution method against it.

We set ``dim=1`` as we want to reproduce the ``rare-features`` experiment from https://arxiv.org/pdf/2106.05303.

Using ``dim=2`` would reproduce the ``rare-time`` experiment.

In [5]:
# Only use the first 10 data points
x = arma.preprocess()["x"][:10]
true_saliency = arma.true_saliency(dim=1)[:10]

### Create attributions using integrated_gradients

We will now get some attributions using the ``integrated_gradients`` method.

The ``forward_func`` associated with Arma is a simple deterministic function: ``get_white_box``.
Please refer to https://arxiv.org/pdf/2106.05303 for more information.

The baselines used here are only zeros.

In [6]:
attr = th.zeros_like(x)
for i, (inputs, saliency) in enumerate(zip(x, true_saliency)):
    explainer = IntegratedGradients(forward_func=arma.get_white_box)
    baseline = inputs * 0
    attr[i] = explainer.attribute(
        inputs,
        baselines=baseline,
        additional_forward_args=(saliency,),
    ).abs()

### Attributions evaluation

Since we know the true attributions, we can evaluate our computed attributions 
using our white-box metrics. For instance, we compute here the ``aur`` (area under recall):

In [7]:
print(f"{aur(attr, true_saliency):.4}")

0.1348


Not a great result! This is close to the results reported in https://arxiv.org/pdf/2106.05303.

There are indeed better methods than integrated_gradients for this task. For more details, 
please refer to our ``experiments/arma`` section.